In [2]:
import datetime
import numpy as np

In [3]:
centerList= [637.40,670.16, 706.20,789.40,761.10, 802.41 ,991.26, 1074.70,1079.80	]

In [6]:
centralWave = 637.4
stepSize = 0.035
waveSteps = 5
cont = "BOTH"
numsums = 16
repeats = 4
exposureTime = 80
LCVRrelaxation = 300
modulations = 4
seconds = 1/1000
cameras =2 

now = datetime.datetime.strftime(datetime.datetime.now(),"%a %b %d %H:%M:%S %Y")
executionTime = (LCVRrelaxation+exposureTime*modulations*numsums)*waveSteps*repeats*cameras*seconds

In [7]:
with open(f"{np.floor(centralWave):.0f}_{str(waveSteps).zfill(2)}wave_{stepSize}step_2beam_{numsums}sums_{repeats}reps_{cont}.rcp","w") as f:
    f.write(f"DATE {now}\n")
    f.write(f"AUTHOR: rcp Generator")
    f.write(f"Expected execution time {executionTime:.1f}\n")
    f.write(f"#DATATYPE BEAM  WAVELENGTH  NUMSUMS\n")
    for repeat in range(repeats):
        for cam in ["RCAM","TCAM"]:
            for i in range(waveSteps):
                f.write(f"DATA {cam}\t{((i-np.floor(waveSteps/2))*stepSize)+centralWave:.2f}\t{cont}\t{numsums}\n")
            f.write("\n")

In [123]:
file = open("eclipsePrep.md")
text = file.readlines()
file.close()

In [107]:
exposureTime = 0
for t in text:
    if "data" in t:
        exposureCount += int(t.split("\t")[-1])*80*4+300

In [109]:
exposureCount/1000/60

92.412